# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [5]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [6]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('GEMINI_API_KEY')

if api_key and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gemini-2.5-flash'
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
gemini = OpenAI(base_url=GEMINI_BASE_URL, api_key=api_key)

API key looks good so far


In [3]:
links = fetch_website_links("https://edwarddonner.com")
links

['https://edwarddonner.com/',
 'https://edwarddonner.com/curriculum/',
 'https://edwarddonner.com/proficient/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://edwarddonner.com/curriculum/',
 'https://edwarddonner.com/2026/02/17/ai-coder-vibe-coder-to-agentic-engineer/',
 'https://edwarddonner.com/2026/02/17/ai-coder-vibe-coder-to-agentic-engineer/',
 'https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/',
 'https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.co

## First step: Have GPT-5-nano figure out which links are relevant

### Use a call to gpt-5-nano to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [7]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [8]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [6]:
print(get_links_user_prompt("https://edwarddonner.com"))


Here is the list of links on the website https://edwarddonner.com -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

https://edwarddonner.com/
https://edwarddonner.com/curriculum/
https://edwarddonner.com/proficient/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://edwarddonner.com/curriculum/
https://edwarddonner.com/2026/02/17/ai-coder-vibe-coder-to-agentic-engineer/
https://edwarddonner.com/2026/02/17/ai-coder-vibe-coder-to-agentic-engineer/
https://edwarddonner.

In [9]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url}...")
    response = gemini.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links.")
    return links

In [8]:
select_relevant_links("https://edwarddonner.com")

{'links': [{'type': 'home page', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'portfolio page', 'url': 'https://edwarddonner.com/curriculum/'},
  {'type': 'skills page', 'url': 'https://edwarddonner.com/proficient/'},
  {'type': 'blog page', 'url': 'https://edwarddonner.com/posts/'},
  {'type': 'affiliated product page',
   'url': 'https://nebula.io/?utm_source=ed&utm_medium=referral'},
  {'type': 'social media page',
   'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'social media page', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'social media page',
   'url': 'https://www.facebook.com/edward.donner.52'}]}

In [9]:
select_relevant_links("https://www.huggingface.co")

{'links': [{'type': 'homepage', 'url': 'https://www.huggingface.co'},
  {'type': 'about page', 'url': 'https://www.huggingface.co/huggingface'},
  {'type': 'enterprise solutions page',
   'url': 'https://www.huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://www.huggingface.co/pricing'},
  {'type': 'blog page', 'url': 'https://www.huggingface.co/blog'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'product offerings page (models)',
   'url': 'https://www.huggingface.co/models'},
  {'type': 'product offerings page (datasets)',
   'url': 'https://www.huggingface.co/datasets'},
  {'type': 'product offerings page (spaces)',
   'url': 'https://www.huggingface.co/spaces'},
  {'type': 'social media profile (GitHub)',
   'url': 'https://github.com/huggingface'},
  {'type': 'social media profile (Twitter)',
   'url': 'https://twitter.com/huggingface'},
  {'type': 'social media profile (LinkedIn)',
   'url': 'https://www.linkedin.c

## Second step: make the brochure!

Assemble all the details into another prompt to GPT-5-nano

In [10]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [12]:
print(fetch_page_and_all_relevant_links("https://huggingface.co"))

Selecting relevant links for https://huggingface.co...
Found 13 relevant links.
## Landing Page:

Hugging Face – The AI community building the future.

Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
NEW
GGML and llama.cpp join Hugging Face 🔥
Try HuggingChat Omni – Chat with AI 💬
Get started with Inference in seconds 🚀
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 2M+ models
Trending on
this week
Models
Qwen/Qwen3.5-35B-A3B
Updated
1 day ago
•
21k
•
395
Qwen/Qwen3.5-397B-A17B
Updated
2 days ago
•
483k
•
1.06k
Qwen/Qwen3.5-27B
Updated
about 18 hours ago
•
6.88k
•
275
zai-org/GLM-5
Updated
12 days ago
•
182k
•
1.55k
Qwen/Qwen3.5-122B-A10B
Updated
1 day ago
•
3.32k
•
233
Browse 2M+ models
Spaces
Running
on
Zero
Featured
1.65k
Qwen Image Multiple Angles 3D Camera
🎥
1.65k
Change the camera angle of a photo with AI
Running
on
Zero
MCP
94

In [11]:
brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# brochure_system_prompt = """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """


In [12]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [15]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co...
Found 6 relevant links.


'\nYou are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages;\nuse this information to build a short brochure of the company in markdown without code blocks.\n\n\n## Landing Page:\n\nHugging Face – The AI community building the future.\n\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nNEW\nGGML and llama.cpp join Hugging Face 🔥\nTry HuggingChat Omni – Chat with AI 💬\nGet started with Inference in seconds 🚀\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 2M+ models\nTrending on\nthis week\nModels\nQwen/Qwen3.5-35B-A3B\nUpdated\n1 day ago\n•\n21k\n•\n395\nQwen/Qwen3.5-397B-A17B\nUpdated\n2 days ago\n•\n483k\n•\n1.06k\nQwen/Qwen3.5-27B\nUpdated\nabout 18 hours ago\n•\n6.88k\n•\n276\nzai-org/GLM-5\nUpdated\n12 days ago\n•\n182k\n•\n1.55k\nQwen/Qwen3.5-122B-A10B\nU

In [13]:
def create_brochure(company_name, url):
    response = gemini.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    return result

In [17]:
create_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co...
Found 11 relevant links.


Hugging Face: The AI Community Building the Future

Welcome to Hugging Face, the leading platform where the global machine learning community collaborates on groundbreaking models, datasets, and applications. We are the home of machine learning, dedicated to making advanced AI accessible, collaborative, and easy to build for everyone.

**What We Offer:**

*   **Models:** Explore and contribute to over 2 million models, from cutting-edge large language models like Qwen to specialized AI.
*   **Datasets:** Access and utilize more than 500,000 datasets, powering diverse AI research and development.
*   **Spaces:** Discover and deploy over 1 million AI applications, showcasing the practical power of machine learning, from image generation to custom speech synthesis.

**Innovate Faster with Hugging Face:**

*   **Collaboration Platform:** Host and collaborate on unlimited public models, datasets, and applications, fostering a dynamic environment for shared progress.
*   **Open Source Advantage:** Leverage the robust Hugging Face open-source stack to accelerate your development.
*   **Multi-modality:** Work across all AI modalities including text, image, video, audio, and even 3D.
*   **Build Your Portfolio:** Share your work with the world, establishing your expertise and building your professional ML presence.
*   **Cutting-edge Tools:** Engage with new features like HuggingChat Omni for AI interaction and get started with inference in seconds. We are constantly evolving, with recent additions like GGML and llama.cpp joining our ecosystem.

**Solutions for Every Innovator:**

*   **For Individuals (PRO Account):** Boost your personal ML journey for just $9/month. Enjoy enhanced storage, increased inference credits, priority access to ZeroGPU compute, Spaces Dev Mode, the ability to publish blog articles, and a distinctive Pro badge.
*   **For Growing Teams (Team Hub):** Instant setup for your team at $20/user/month. Empower your group with a collaborative environment designed for rapid development.
*   **For Enterprises (Enterprise Hub):** Scale your organization with the world’s leading AI platform. Contact our sales team for flexible contracts that provide enterprise-grade security, advanced access controls, dedicated priority support, and robust features like Single Sign-On (SSO), audit logs, granular resource management, advanced compute options including ZeroGPU quota boosts, private storage, and comprehensive billing controls.

Join the most forward-thinking AI organizations and individuals who are shaping the future of artificial intelligence with Hugging Face. Whether you're a developer, a researcher, a startup, or a large corporation, Hugging Face provides the tools, community, and infrastructure to create, discover, and collaborate on ML better.

Explore AI Apps, Browse Models, or Get Started with Inference Today!

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [18]:
def stream_brochure(company_name, url):
    stream = gemini.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [19]:
stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co...
Found 8 relevant links.


**Hugging Face: The AI Community Building the Future**

Welcome to Hugging Face, the leading platform where the global machine learning community collaborates to create, discover, and deploy the next generation of artificial intelligence. We are dedicated to making machine learning accessible, collaborative, and powerful for everyone, serving as the home of machine learning.

**What We Offer:**

Hugging Face provides an unparalleled ecosystem for ML development and deployment:

*   **Models:** Access and contribute to an expansive library of over 2 million state-of-the-art machine learning models, covering diverse applications from large language models to specialized tasks.
*   **Datasets:** Explore and share from over 500,000 datasets, providing the essential fuel for research and innovation.
*   **Spaces (AI Applications):** Discover and deploy over 1 million interactive AI applications. These "Spaces" allow you to experiment with AI instantly, from image generation and video creation to custom speech synthesis.
*   **Collaboration Platform:** Host and collaborate on unlimited public models, datasets, and applications, fostering a dynamic environment for shared progress and innovation.
*   **Open Source Advantage:** Leverage the robust Hugging Face open-source stack to accelerate your ML projects and move faster with cutting-edge tools.
*   **Multi-Modal Support:** Work across all major modalities including text, image, video, audio, and even 3D, pushing the boundaries of what AI can achieve. We continuously integrate new technologies like GGML and llama.cpp, and offer advanced features such as HuggingChat Omni and rapid inference.

**Who We Serve:**

*   **The Global AI Community:** Whether you're a student, researcher, developer, or enthusiast, Hugging Face empowers you to build your ML portfolio, share your work, and contribute to the collective intelligence of AI. Join millions in shaping the future!
*   **Enterprises & Organizations:** Scale your AI initiatives with the **Team & Enterprise Hub**. Benefit from enterprise-grade security, robust access controls, dedicated support, Single Sign-On (SSO) integration, regional data management, and comprehensive audit logs. Build and deploy AI securely and efficiently within your organization.

**Our Culture:**

At our core, Hugging Face champions community, collaboration, and open innovation. We believe that by working together, we can unlock the full potential of AI and make a positive impact on the world. Our platform is designed to help you create, discover, and collaborate on ML better, fostering an environment where professionals can build their ML portfolio and contribute to the broader AI ecosystem.

**Join Us:**

Explore the future of AI. Whether you're looking to browse models, create an app, contribute a dataset, or scale your organizational AI, Hugging Face is your home for machine learning. Get started with inference in seconds or connect with our sales team for tailored enterprise solutions.

In [1]:
translation_system_prompt = """"
        You are a professional translator that can translate a company brochure into different languages.
        You should focus on translating the content naturally and in a way that is appropriate for the target language,
        while maintaining the original meaning and tone of the brochure. You will receive the brochure in markdown format,
        and you should respond with the translated brochure also in markdown format, without code blocks.
    """

In [2]:
def create_translation_user_prompt(company, url, target_language):
    user_prompt = f"""Please translate the following company brochure into {target_language}. Focus on
        translating the content naturally and in a way that is appropriate for the target language,
        while maintaining the original meaning and tone of the brochure. Respond in markdown format without code blocks.
        Here is the brochure to translate:
        
    """
    user_prompt += create_brochure(company, url)
    return user_prompt

In [3]:
def translated_brochure(company, url, target_language):
    response = gemini.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": translation_system_prompt},
            {"role": "user", "content": create_translation_user_prompt(company, url, target_language)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [14]:
translated_brochure("HuggingFace", "https://huggingface.co", "Hungarian")

Selecting relevant links for https://huggingface.co...
Found 10 relevant links.


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


**Hugging Face: A Jövőt Építő MI Közösség**

Üdvözöljük a Hugging Face-nél, a vezető platformon, ahol a globális gépi tanulási közösség együttműködik a mesterséges intelligencia jövőjének építésében. Mi vagyunk a gépi tanulás otthona, elkötelezettek vagyunk amellett, hogy a legmodernebb MI-t hozzáférhetővé, együttműködésivé és gyorssá tegyük.

**Amit Kínálunk:**

*   **Modellek:** Fedezzen fel és használjon több mint 2 millió modellt, amelyek szöveges, képi, videós, hang- és akár 3D-s modalitásokban is elérhetőek. Fedezze fel a legújabb innovációkat, és integrálja könnyedén a hatékony MI-t projektjeibe.
*   **Adathalmazok:** Hozzáférés több mint 500 000 adathalmaz hatalmas tárházához, amelyek az alapvető adatbázist biztosítják gépi tanulási modelljei képzéséhez és finomhangolásához.
*   **Spaces:** Engedje szabadjára kreativitását több mint 1 millió MI alkalmazással és demóval. Tartson, fedezzen fel és osszon meg interaktív MI-élményeket – a képek és videók generálásától az egyedi beszédszintézisig és azon túl.
*   **Együttműködési Platform:** Alkoss, fedezz fel és működj együtt hatékonyabban a gépi tanulásban. A Hugging Face lehetővé teszi, hogy korlátlan számú nyilvános modellen, adathalmazon és alkalmazáson dolgozzon és osztozzon, dinamikus környezetet teremtve az innovációhoz.
*   **Nyílt Forráskódú Technológiai Megoldások:** Gyorsuljon fel a robusztus Hugging Face nyílt forráskódú technológiájával, amelyet a fejlesztési és kutatási munkafolyamatok felgyorsítására terveztek.
*   **Következtetés Másodpercek Alatt:** Indítsa el az MI következtetést gyorsan és hatékonyan, azonnal életre keltve modelljeit.
*   **HuggingChat Omni:** Vegye igénybe MI chatbotunkat, felfedezve a legújabbakat a konverzációs MI területén.

**Kiknek Szólunk?**

*   **Ügyfelek:** Egyedi fejlesztőktől a nagyvállalatokig a Hugging Face biztosítja az eszközöket, erőforrásokat és közösséget az MI fejlesztésének és bevezetésének felgyorsításához.
*   **Befektetők:** Csatlakozzon hozzánk a nyílt forráskódú MI forradalom motorjaként. Gyorsan növekvő közösségünk és átfogó platformunk az MI innováció élvonalában áll.
*   **Leendő Munkatársak:** Legyen része egy vibráló, aktív közösségnek, amely az MI fejlesztésének szenteli magát. Bár itt konkrét állásajánlatok nincsenek feltüntetve, egy ellenőrzött vállalat vagyunk, amelynek csapata elkötelezett az MI jövőjének építése iránt, egyedülálló lehetőséget kínálva, hogy tízezrek gépi tanulás iránti rajongóval együtt hozzájáruljon nagy hatású projektekhez. Építse fel gépi tanulási portfólióját, és ossza meg munkáját a világgal.

Csatlakozzon a Hugging Face közösséghez még ma, és járuljon hozzá a mesterséges intelligencia következő generációjának alakításához.

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>